In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch as ch
from torch.utils.data import Dataset

import torchvision
import torchvision.transforms as transforms

import os

import time
from models import ResNet18
from advertorch.attacks import L2PGDAttack

In [2]:
learning_rate = 0.1
file_name = 'basic_training_robust_dataset'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform_train = transforms.Compose([
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])

class TensorDataset(Dataset):
    def __init__(self, *tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        im, targ = tuple(tensor[index] for tensor in self.tensors)
        if self.transform:
            real_transform = transforms.Compose([
                transforms.ToPILImage(),
                self.transform
            ])
            im = real_transform(im)
        return im, targ

    def __len__(self):
        return self.tensors[0].size(0)

data_path = "d_robust_CIFAR/"
train_data = ch.cat(ch.load(os.path.join(data_path, f"CIFAR_ims")))
train_labels = ch.cat(ch.load(os.path.join(data_path, f"CIFAR_lab")))
train_dataset = TensorDataset(train_data, train_labels, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified


In [3]:
net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

adversary = L2PGDAttack(net, loss_fn=nn.CrossEntropyLoss(), eps=0.25, nb_iter=100, eps_iter=0.01, rand_init=True, clip_min=0.0, clip_max=1.0, targeted=False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

In [4]:
def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 10 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)

In [5]:
def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    benign_loss = 0
    adv_loss = 0
    benign_correct = 0
    adv_correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        benign_loss += loss.item()

        _, predicted = outputs.max(1)
        benign_correct += predicted.eq(targets).sum().item()

        if batch_idx % 10 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign test loss:', loss.item())

        adv = adversary.perturb(inputs, targets)
        adv_outputs = net(adv)
        loss = criterion(adv_outputs, targets)
        adv_loss += loss.item()

        _, predicted = adv_outputs.max(1)
        adv_correct += predicted.eq(targets).sum().item()

        if batch_idx % 10 == 0:
            print('Current adversarial test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current adversarial test loss:', loss.item())

    print('\nTotal benign test accuarcy:', 100. * benign_correct / total)
    print('Total adversarial test Accuarcy:', 100. * adv_correct / total)
    print('Total benign test loss:', benign_loss)
    print('Total adversarial test loss:', adv_loss)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [6]:
for epoch in range(0, 200):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    #test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.125
Current benign train loss: 2.3160736560821533

Current batch: 10
Current benign train accuracy: 0.125
Current benign train loss: 5.190974235534668

Current batch: 20
Current benign train accuracy: 0.046875
Current benign train loss: 3.223655939102173

Current batch: 30
Current benign train accuracy: 0.0546875
Current benign train loss: 2.56705379486084

Current batch: 40
Current benign train accuracy: 0.140625
Current benign train loss: 2.2814841270446777

Current batch: 50
Current benign train accuracy: 0.0703125
Current benign train loss: 2.382124900817871

Current batch: 60
Current benign train accuracy: 0.125
Current benign train loss: 2.2685041427612305

Current batch: 70
Current benign train accuracy: 0.109375
Current benign train loss: 2.354434013366699

Current batch: 80
Current benign train accuracy: 0.140625
Current benign train loss: 2.1287925243377686

Current batch: 90
Current benign train accuracy:


Current batch: 370
Current benign train accuracy: 0.53125
Current benign train loss: 1.320539116859436

Current batch: 380
Current benign train accuracy: 0.578125
Current benign train loss: 1.2123702764511108

Current batch: 390
Current benign train accuracy: 0.525
Current benign train loss: 1.4005134105682373

Total benign train accuarcy: 47.812
Total benign train loss: 562.6526744365692

[ Train epoch: 2 ]

Current batch: 0
Current benign train accuracy: 0.515625
Current benign train loss: 1.3084170818328857

Current batch: 10
Current benign train accuracy: 0.546875
Current benign train loss: 1.1531381607055664

Current batch: 20
Current benign train accuracy: 0.546875
Current benign train loss: 1.2675633430480957

Current batch: 30
Current benign train accuracy: 0.515625
Current benign train loss: 1.3285236358642578

Current batch: 40
Current benign train accuracy: 0.6015625
Current benign train loss: 1.22389554977417

Current batch: 50
Current benign train accuracy: 0.5703125
Curr


Current batch: 330
Current benign train accuracy: 0.703125
Current benign train loss: 0.8059946894645691

Current batch: 340
Current benign train accuracy: 0.7265625
Current benign train loss: 0.8497375845909119

Current batch: 350
Current benign train accuracy: 0.734375
Current benign train loss: 0.722388744354248

Current batch: 360
Current benign train accuracy: 0.7421875
Current benign train loss: 0.7444250583648682

Current batch: 370
Current benign train accuracy: 0.7109375
Current benign train loss: 0.6802464723587036

Current batch: 380
Current benign train accuracy: 0.6953125
Current benign train loss: 0.8250921368598938

Current batch: 390
Current benign train accuracy: 0.7125
Current benign train loss: 0.760513186454773

Total benign train accuarcy: 68.798
Total benign train loss: 341.3352583050728

[ Train epoch: 4 ]

Current batch: 0
Current benign train accuracy: 0.78125
Current benign train loss: 0.6777499914169312

Current batch: 10
Current benign train accuracy: 0.796


Current batch: 290
Current benign train accuracy: 0.8125
Current benign train loss: 0.5545961856842041

Current batch: 300
Current benign train accuracy: 0.765625
Current benign train loss: 0.7379340529441833

Current batch: 310
Current benign train accuracy: 0.796875
Current benign train loss: 0.5442806482315063

Current batch: 320
Current benign train accuracy: 0.796875
Current benign train loss: 0.5808265805244446

Current batch: 330
Current benign train accuracy: 0.78125
Current benign train loss: 0.5572057962417603

Current batch: 340
Current benign train accuracy: 0.7734375
Current benign train loss: 0.6540535092353821

Current batch: 350
Current benign train accuracy: 0.8515625
Current benign train loss: 0.43075114488601685

Current batch: 360
Current benign train accuracy: 0.8046875
Current benign train loss: 0.4306660294532776

Current batch: 370
Current benign train accuracy: 0.796875
Current benign train loss: 0.6184571981430054

Current batch: 380
Current benign train accu


Current batch: 250
Current benign train accuracy: 0.875
Current benign train loss: 0.3723933696746826

Current batch: 260
Current benign train accuracy: 0.8515625
Current benign train loss: 0.4044782519340515

Current batch: 270
Current benign train accuracy: 0.84375
Current benign train loss: 0.35416489839553833

Current batch: 280
Current benign train accuracy: 0.8359375
Current benign train loss: 0.49436140060424805

Current batch: 290
Current benign train accuracy: 0.8828125
Current benign train loss: 0.28983959555625916

Current batch: 300
Current benign train accuracy: 0.859375
Current benign train loss: 0.3020385205745697

Current batch: 310
Current benign train accuracy: 0.8515625
Current benign train loss: 0.358335018157959

Current batch: 320
Current benign train accuracy: 0.8671875
Current benign train loss: 0.39323320984840393

Current batch: 330
Current benign train accuracy: 0.921875
Current benign train loss: 0.2349407821893692

Current batch: 340
Current benign train a

...


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0005849961307831109

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0006762327393516898

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.000864935340359807

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0007626977749168873

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0007001390913501382

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0010149118024855852

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0014075167709961534

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0009510755771771073

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0011701174080371857

Current batch: 100
Current benign train accuracy: 1.0
Current benign trai


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0013818277511745691

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0009428835473954678

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0014288839884102345

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0007297074189409614

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0009080867166630924

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0006763232522644103

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0010344249894842505

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0012274645268917084

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0016516197938472033

Current batch: 100
Current benign train accuracy: 1.0
Current benign tra


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0009316573850810528

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0007912438595667481

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0007268273620866239

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0012131166877225041

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0012495709815993905

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0008315135492011905

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0019149231957271695

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0008980815182439983

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0007685763412155211

Current batch: 100
Current benign train accuracy: 1.0
Current benign tra


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0010542008094489574

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0008298866450786591

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.001255555311217904

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0009447070769965649

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0009340575779788196

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0008674468263052404

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0008526469464413822

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0010268923360854387

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0008962384890764952

Current batch: 100
Current benign train accuracy: 1.0
Current benign trai


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0010468843393027782

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0011158892884850502

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0011641366872936487

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0011495387880131602

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.000904326094314456

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0009494165424257517

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0011899981182068586

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0006461413577198982

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0021044714376330376

Current batch: 100
Current benign train accuracy: 1.0
Current benign trai

In [7]:
test(epoch)


[ Test epoch: 199 ]

Current batch: 0
Current benign test accuracy: 0.82
Current benign test loss: 0.7186563014984131
Current adversarial test accuracy: 0.3
Current adversarial test loss: 3.79893159866333

Current batch: 10
Current benign test accuracy: 0.73
Current benign test loss: 1.0755268335342407
Current adversarial test accuracy: 0.37
Current adversarial test loss: 3.8852109909057617

Current batch: 20
Current benign test accuracy: 0.75
Current benign test loss: 0.9439548254013062
Current adversarial test accuracy: 0.33
Current adversarial test loss: 4.361649513244629

Current batch: 30
Current benign test accuracy: 0.82
Current benign test loss: 0.6866134405136108
Current adversarial test accuracy: 0.32
Current adversarial test loss: 3.604046106338501

Current batch: 40
Current benign test accuracy: 0.85
Current benign test loss: 0.49680861830711365
Current adversarial test accuracy: 0.35
Current adversarial test loss: 3.544793128967285

Current batch: 50
Current benign test a

In [8]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0002)
for epoch in range(200, 250):
    train(epoch)


[ Train epoch: 200 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0008726849919185042

Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0011476755607873201

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0008728919201530516

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0008452764013782144

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0008205946651287377

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0006833854131400585

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0008279865724034607

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0009972886182367802

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0013399501331150532

Current batch: 90
Current benign train accuracy: 1.


Current batch: 380
Current benign train accuracy: 1.0
Current benign train loss: 0.0008837330969981849

Current batch: 390
Current benign train accuracy: 1.0
Current benign train loss: 0.0010760993463918567

Total benign train accuarcy: 100.0
Total benign train loss: 0.37962731398874894

[ Train epoch: 202 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0007047982071526349

Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0006337295635603368

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0006715378258377314

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0007772587705403566

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0008966124150902033

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0008894114871509373

Current batch: 60
Current benign train accuracy: 1.0
Current benign tra


Current batch: 360
Current benign train accuracy: 1.0
Current benign train loss: 0.0012861621798947453

Current batch: 370
Current benign train accuracy: 1.0
Current benign train loss: 0.0007976031629368663

Current batch: 380
Current benign train accuracy: 1.0
Current benign train loss: 0.0007432460552081466

Current batch: 390
Current benign train accuracy: 1.0
Current benign train loss: 0.0008095150697045028

Total benign train accuarcy: 100.0
Total benign train loss: 0.3741050589014776

[ Train epoch: 204 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0009069652878679335

Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.001160734798759222

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0010223411954939365

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.000842770969029516

Current batch: 40
Current benign train accuracy: 1.0
Current benign trai


Current batch: 340
Current benign train accuracy: 1.0
Current benign train loss: 0.0007496271282434464

Current batch: 350
Current benign train accuracy: 1.0
Current benign train loss: 0.0010346267372369766

Current batch: 360
Current benign train accuracy: 1.0
Current benign train loss: 0.0007696680258959532

Current batch: 370
Current benign train accuracy: 1.0
Current benign train loss: 0.00124267372302711

Current batch: 380
Current benign train accuracy: 1.0
Current benign train loss: 0.0014649438671767712

Current batch: 390
Current benign train accuracy: 1.0
Current benign train loss: 0.0036189532838761806

Total benign train accuarcy: 100.0
Total benign train loss: 0.37311598082305863

[ Train epoch: 206 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0009080374147742987

Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0007730508805252612

Current batch: 20
Current benign train accuracy: 1.0
Current benign t


Current batch: 320
Current benign train accuracy: 1.0
Current benign train loss: 0.0008964731241576374

Current batch: 330
Current benign train accuracy: 1.0
Current benign train loss: 0.0010407078079879284

Current batch: 340
Current benign train accuracy: 1.0
Current benign train loss: 0.000621309329289943

Current batch: 350
Current benign train accuracy: 1.0
Current benign train loss: 0.0009692201856523752

Current batch: 360
Current benign train accuracy: 1.0
Current benign train loss: 0.0007138709770515561

Current batch: 370
Current benign train accuracy: 1.0
Current benign train loss: 0.001087578129954636

Current batch: 380
Current benign train accuracy: 1.0
Current benign train loss: 0.0008312542922794819

Current batch: 390
Current benign train accuracy: 1.0
Current benign train loss: 0.0008242543553933501

Total benign train accuarcy: 100.0
Total benign train loss: 0.37711553723784164

[ Train epoch: 208 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign

...


Current batch: 390
Current benign train accuracy: 1.0
Current benign train loss: 0.0013111222069710493

Total benign train accuarcy: 100.0
Total benign train loss: 0.3794548384612426

[ Train epoch: 241 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0008228464284911752

Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0009307353757321835

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0014310223050415516

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0008569558849558234

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0008518164395354688

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0007630920154042542

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0010412882547825575

Current batch: 70
Current benign train accuracy: 1.0
Current benign train


Current batch: 370
Current benign train accuracy: 1.0
Current benign train loss: 0.0011932639172300696

Current batch: 380
Current benign train accuracy: 1.0
Current benign train loss: 0.000974764465354383

Current batch: 390
Current benign train accuracy: 1.0
Current benign train loss: 0.0026286740321666002

Total benign train accuarcy: 100.0
Total benign train loss: 0.3716406458988786

[ Train epoch: 243 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0007526781992055476

Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0006321474211290479

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0008378023048862815

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0007171271136030555

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.000845474423840642

Current batch: 50
Current benign train accuracy: 1.0
Current benign train


Current batch: 350
Current benign train accuracy: 1.0
Current benign train loss: 0.0009128483943641186

Current batch: 360
Current benign train accuracy: 1.0
Current benign train loss: 0.0011788621777668595

Current batch: 370
Current benign train accuracy: 1.0
Current benign train loss: 0.0008414281764999032

Current batch: 380
Current benign train accuracy: 1.0
Current benign train loss: 0.0009929739171639085

Current batch: 390
Current benign train accuracy: 1.0
Current benign train loss: 0.0009471207740716636

Total benign train accuarcy: 100.0
Total benign train loss: 0.37722211866639555

[ Train epoch: 245 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0012428286718204618

Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0009036945994012058

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0008201063028536737

Current batch: 30
Current benign train accuracy: 1.0
Current benign 


Current batch: 330
Current benign train accuracy: 1.0
Current benign train loss: 0.0009405405726283789

Current batch: 340
Current benign train accuracy: 1.0
Current benign train loss: 0.0007601951947435737

Current batch: 350
Current benign train accuracy: 1.0
Current benign train loss: 0.0008660773746669292

Current batch: 360
Current benign train accuracy: 1.0
Current benign train loss: 0.0008114253869280219

Current batch: 370
Current benign train accuracy: 1.0
Current benign train loss: 0.0011968674371019006

Current batch: 380
Current benign train accuracy: 1.0
Current benign train loss: 0.001038223272189498

Current batch: 390
Current benign train accuracy: 1.0
Current benign train loss: 0.0010572387836873531

Total benign train accuarcy: 100.0
Total benign train loss: 0.3757233933429234

[ Train epoch: 247 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0009403894073329866

Current batch: 10
Current benign train accuracy: 1.0
Current benign 


Current batch: 310
Current benign train accuracy: 1.0
Current benign train loss: 0.0008525184821337461

Current batch: 320
Current benign train accuracy: 1.0
Current benign train loss: 0.0014871410094201565

Current batch: 330
Current benign train accuracy: 1.0
Current benign train loss: 0.0006412140792235732

Current batch: 340
Current benign train accuracy: 1.0
Current benign train loss: 0.001107176416553557

Current batch: 350
Current benign train accuracy: 1.0
Current benign train loss: 0.0007976788328960538

Current batch: 360
Current benign train accuracy: 1.0
Current benign train loss: 0.0007717880071140826

Current batch: 370
Current benign train accuracy: 1.0
Current benign train loss: 0.0012079619336873293

Current batch: 380
Current benign train accuracy: 1.0
Current benign train loss: 0.0010118786012753844

Current batch: 390
Current benign train accuracy: 1.0
Current benign train loss: 0.0016643155831843615

Total benign train accuarcy: 100.0
Total benign train loss: 0.37

In [9]:
test(epoch)


[ Test epoch: 249 ]

Current batch: 0
Current benign test accuracy: 0.82
Current benign test loss: 0.700284481048584
Current adversarial test accuracy: 0.31
Current adversarial test loss: 3.8037827014923096

Current batch: 10
Current benign test accuracy: 0.74
Current benign test loss: 1.0594165325164795
Current adversarial test accuracy: 0.38
Current adversarial test loss: 3.878237724304199

Current batch: 20
Current benign test accuracy: 0.74
Current benign test loss: 0.945330023765564
Current adversarial test accuracy: 0.33
Current adversarial test loss: 4.397638320922852

Current batch: 30
Current benign test accuracy: 0.82
Current benign test loss: 0.6879650950431824
Current adversarial test accuracy: 0.34
Current adversarial test loss: 3.5778591632843018

Current batch: 40
Current benign test accuracy: 0.85
Current benign test loss: 0.48312637209892273
Current adversarial test accuracy: 0.35
Current adversarial test loss: 3.535510540008545

Current batch: 50
Current benign test 